<a href="https://colab.research.google.com/github/palaceIA/CICS_Project/blob/main/src/notebooks/distilbert/distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning Distilbert

Esse jupyter faz parte na nossa pesquisa para o Congresso Internacional (CICS) , aqui você ira encontrar nosso experimento referente ao modelo DistilBert e os nossos devidos resultados. O dataset utlizado foi o "dair-ai/emotion" disponivel no HuggingFace .

## DistilBert-base-uncased

DistilBERT é um modelo de transformadores, menor e mais rápido que o BERT, que foi pré-treinado no mesmo corpus de forma autossupervisionada, usando o modelo base do BERT como professor. Isso significa que ele foi pré-treinado apenas nos textos brutos, sem humanos rotulando-os de forma alguma (é por isso que ele pode usar muitos dados disponíveis publicamente) com um processo automático para gerar entradas e rótulos desses textos usando o modelo base do BERT. Mais precisamente, ele foi pré-treinado com três objetivos:

Perda de destilação: o modelo foi treinado para retornar as mesmas probabilidades que o modelo base BERT.
Modelagem de linguagem mascarada (MLM): isso faz parte da perda de treinamento original do modelo base BERT. Ao pegar uma frase, o modelo mascara aleatoriamente 15% das palavras na entrada e então executa a frase mascarada inteira através do modelo e tem que prever as palavras mascaradas. Isso é diferente das redes neurais recorrentes tradicionais (RNNs) que geralmente veem as palavras uma após a outra, ou de modelos autorregressivos como GPT que mascaram internamente os tokens futuros. Ele permite que o modelo aprenda uma representação bidirecional da frase.
Perda de incorporação de cosseno: o modelo também foi treinado para gerar estados ocultos o mais próximo possível do modelo base BERT.
Dessa forma, o modelo aprende a mesma representação interna da língua inglesa que o modelo do professor, sendo mais rápido para inferência ou tarefas posteriores.

## Treinamento do modelo

O DistilBERT foi pré-treinado com os mesmos dados do BERT, que é o BookCorpus , um conjunto de dados que consiste em 11.038 livros não publicados e a Wikipédia em inglês (excluindo listas, tabelas e cabeçalhos).

In [ ]:
# Instalando os modulos
!pip install torch transformers datasets matplotlib numpy scikit-learn pandas

### Carregando os modulos

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    AutoTokenizer,
    AutoModel , 
    Trainer
)
from sklearn.metrics import (
    accuracy_score ,
    f1_score ,
    classification_report , 
    roc_curve , 
    auc
)
from scipy.special import softmax
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

### Carregando o dataset

In [ ]:
id_data = "dair-ai/emotion"
dataset = load_dataset(id_data)

### Entendendo os dados

In [ ]:
# Estrutura do dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
# Total de classes do dataset
classes = dataset['train'].features['label'].names
classes

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
# Alterando o formato do dataset para um tipo pandas
dataset.set_format(type='pandas')
df_pandas = dataset['train'][:]
df_pandas.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [ ]:
# Criando uma tabela para cada classe correspondente
df_pandas['label_name'] = df_pandas['label'].apply(lambda x : classes[x])
df_pandas.head()

,text,label,label_name
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger


In [ ]:
# Verificando o balanceamento das classes
total_classes = df_pandas['label_name'].value_counts()
total_classes

,count
label_name,
joy,5362
sadness,4666
anger,2159
fear,1937
love,1304
surprise,572


In [ ]:
# Resetando o formato original dos dados
dataset.reset_format()

### Carregando o tokenizador do modelo

In [ ]:
id_model = 'distilbert-base-uncased'
tokenizador = AutoTokenizer.from_pretrained(id_model)

In [ ]:
# passaremos a quantidade de batchs dos dados
# Para aplicar essas a tokenizaço de todos os dados
# basta usar o metodo map()
# Função para tokenizar o dataset
def tokenizador_lote(batch):
    temp = tokenizador(
        batch['text'],  # Aqui, 'batch' deve ser um dicionário com uma chave 'text'
        padding=True,
        truncation=True,
    )
    return temp

### Tokenizando dados

In [ ]:
dataset_tokenizado = dataset.map(
    tokenizador_lote,
    batched = True ,
    batch_size=None
)

### Carregando modelo

In [ ]:
model= AutoModel.from_pretrained(id_model)

In [ ]:
model

### Configurações iniciais para o ajuste fino

In [ ]:
# Armazenando numero de classes
numero_classes = len(classes)
# Inicializando plataforma CUDA
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
model = AutoModelForSequenceClassification.from_pretrained(
    id_model , num_labels = numero_classes
)

In [ ]:
device

### Configurações do treinamento

In [ ]:
# Tamanho do lote
batch_size = 15
model_name = 'distilbert-base-uncased-emotions'

training_args = TrainingArguments(
    output_dir=model_name ,
    num_train_epochs=4 ,
    learning_rate=1e-5,
    per_device_train_batch_size= batch_size ,
    per_device_eval_batch_size=batch_size ,
    weight_decay=0.01,
    eval_strategy='epoch' ,
    disable_tqdm=False

)

### Computação de métricas

In [ ]:
def computer_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels,preds,average='weighted')
    acc = accuracy_score(labels,preds)
    return {"acurracy : " : acc , "f1" : f1}

### Treinamento

In [ ]:
trainer = Trainer(
    model = model ,
    args = training_args ,
    compute_metrics = computer_metrics ,
    train_dataset = dataset_tokenizado['train'] ,
    eval_dataset= dataset_tokenizado['validation'] ,
    tokenizer = tokenizador
)

In [ ]:
trainer.train()

### Avaliando modelo

In [ ]:
pred_ouptus = trainer.predict(
    dataset_tokenizado['test']
)
pred_ouptus.metrics

In [ ]:
y_pred = np.argmax(
    pred_ouptus.predictions,axis=1
)
y_test = dataset_tokenizado['test'][:]['label']

In [ ]:
report_dict = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
print(report_dict)

In [ ]:
report_df = pd.DataFrame(report_dict).transpose()
report_df = report_df.round(4)

# Exibe a tabela
print(report_df)

In [ ]:
y_pred_proba = softmax(y_pred, axis=1)  # importante!

In [ ]:
# Binariza os labels para ROC multiclasse
y_test_bin = label_binarize(y_test, classes=list(range(len(classes))))
# y_pred_proba: array com probabilidade de cada classe (shape: [N, 6])

# Calcula curva ROC e AUC para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plota as curvas ROC
plt.figure(figsize=(10, 8))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], lw=2,
             label=f"{classes[i]} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], "k--", lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC por Classe")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
data = []
for i, cls in enumerate(classes):
    # Protege contra índice fora do range se houver poucos thresholds
    fpr_value = fpr[i][1] if len(fpr[i]) > 1 else fpr[i][0]
    tpr_value = tpr[i][1] if len(tpr[i]) > 1 else tpr[i][0]

    data.append({
        "Classe": cls,
        "AUC": round(roc_auc[i], 4),
        "FPR (Exemplo)": round(fpr_value, 4),
        "TPR (Exemplo)": round(tpr_value, 4)
    })

roc_df = pd.DataFrame(data)
print(roc_df)